# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera. **(A to już znamy!!!)**

## Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [129]:
import math, random

def prime_naive(n):
    if n < 2:
        return False

    m = math.isqrt(n)
    for i in range(2, m + 1):
        if n % i == 0:
            return False

    return True

small_prime_set = set()
for i in range(2, 2000):
    if prime_naive(i):
        small_prime_set.add(i)

In [130]:
def miller_rabin_test(n, iter):
    d = n - 1
    s = 0

    while d % 2 == 0:
        d //= 2 
        s += 1

    def check_a(n, a, d, s):
        x = pow(a, d, mod=n) 

        if x == 1:
            return True
        
        for _ in range(s):
            if x == n - 1:
                return True
            x = pow(x, 2, n)
            
        return False

    for _ in range(iter):
        a = random.randrange(1, n)
        
        if not check_a(n, a, d, s):
            return False
        
    return True

In [131]:
def generate_n_bit_num(n_bits):
    num = random.getrandbits(n_bits)
    num |= (1 << (n_bits - 1))
    num |= 1
    return num

In [132]:
# schneier prime
def generatePrime(n_bits):
    while True:
        n = generate_n_bit_num(n_bits)
        
        composite = False
        for p in small_prime_set:
            if n % p == 0:
                composite = True
                break
            
        if not composite and miller_rabin_test(n, 5):
            return n

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$ **(potrzebujemy rozszerzonego algorytmu Euklidesa!!!)**
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [133]:
def extended_euclides(a, b):
    r_prev = a
    r_cur = b
    s_prev = 1
    s_cur = 0
    t_prev = 0
    t_cur = 1

    while r_cur != 0:
        q = r_prev // r_cur
        r_next = r_prev - q * r_cur
        s_next = s_prev - q * s_cur 
        t_next = t_prev - q * t_cur
        
        r_prev = r_cur
        r_cur = r_next
        s_prev = s_cur
        s_cur = s_next
        t_prev = t_cur
        t_cur = t_next
    
    return r_prev, s_prev, t_prev

def mod_inverse(a, n):
    r, s, t = extended_euclides(a, n)

    if r != 1:
        return None
    else:
        a_inverse = s % n
        return a_inverse

In [134]:
import random, sys, os

def generateKey(keySize):
    p = generatePrime(keySize)
    q = generatePrime(keySize)
    n = p * q   
    p1q1 = (p - 1) * (q - 1)
    e = 65537 # generate_n_bit_num(keySize) 
    while math.gcd(p1q1, e) != 1:
        e = generate_n_bit_num(keySize)
    
    d = mod_inverse(e, p1q1)
    
    publicKey = (n, e)
    privateKey = (n, d)

    return (publicKey, privateKey)

def makeKeyFiles(keySize):
    public, private = generateKey(keySize)
    with open('public.txt', 'w') as file:
        file.write(f"{public[0]}\n{public[1]}")
    with open('private.txt', 'w') as file:
        file.write(f"{private[0]}\n{private[1]}")
  
print('Generujemy klucze publiczny i prywatny')
makeKeyFiles(16)
(n1, e1), (_, d1) = generateKey(16)

Generujemy klucze publiczny i prywatny


## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [135]:
def encrypt(message, modulus, exp):
    message_ascii = [ord(c) for c in message]
    message_encrypted = [pow(c, exp, modulus) for c in message_ascii]
    return message_encrypted

msg = "test"
encrypted_msg = encrypt(msg, n1, e1)

### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [136]:
def decrypt(message_encrypted, modulus, exp):
    message_ascii = [chr(pow(c, exp, modulus)) for c in message_encrypted]
    return ('').join(message_ascii)

decrypted_msg = decrypt(encrypted_msg, n1, d1)

print(msg, encrypted_msg, decrypted_msg)

test [50937415, 145269733, 1092666068, 50937415] test


## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy) - zweryfikuj jak na wydajnosć wpływa zastosowanie różnych sposobów potęgowania dostępnych w Python i jego bibliotekach.  
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


### Algorytm szybkiego potęgowania 
1. Zwykłe potęgowanie $n^{exp}$: $exp$ mnożeń 
2. Algorytm szybkiego potęgowania: część mnożeń zastępujemy podnoszeniem do kwadratu (_squaring_).
    __Skąd mamy wiedzieć kiedy mnożyć, a kiedy potęgować?__

In [137]:

def fastModularExponentation(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m

    